In [142]:
# Dependencies
import requests
import csv
import json
import pandas as pd
from pprint import pprint

In [71]:
# API: https://covid19api.com/
# Covid-19 API request to retrieve the country slug for per country (Australia) requests
url = "https://api.covid19api.com/countries"
response = requests.get(url)

for response in range(len(response_json)):
    if response_json[response]['Country'] == "Australia":
        aus_response = response_json[response]
    
print(aus_response)    

{'Country': 'Australia', 'Slug': 'australia', 'ISO2': 'AU'}


In [72]:
# Covid-19 API summary request for Australia
# Includes new cases and total cases, new deaths and total deaths

# Build URL using the Covid-19 api
url = "https://api.covid19api.com/summary"

response = requests.get(url)
response_json = response.json()

country_response = response_json['Countries']

for response in range(len(country_response)):
    if country_response[response]['Country'] == "Australia":
        au_response = country_response[response]
    

pprint(au_response)   

{'Country': 'Australia',
 'CountryCode': 'AU',
 'Date': '2020-10-01T12:57:04Z',
 'NewConfirmed': 18,
 'NewDeaths': 2,
 'NewRecovered': 32,
 'Premium': {},
 'Slug': 'australia',
 'TotalConfirmed': 27096,
 'TotalDeaths': 888,
 'TotalRecovered': 24782}


In [107]:
# Covid-19 API request for Australia, case status includes active, confirmed, deaths, recovered
# Month: March

# Build URL using the Covid-19 api
url = "https://api.covid19api.com/country/australia?from=2020-03-01T00:00:00Z&to=2020-03-31T00:00:00Z"

# Make request
response = requests.get(url)

# Convert to json
cases_response = response.json()

# Use list comprehension to get case numbers
confirmed = [case_number["Confirmed"] for case_number in cases_response]
active = [case_number["Active"] for case_number in cases_response]
deaths = [case_number["Deaths"] for case_number in cases_response]
state = [case_number["Province"] for case_number in cases_response]
date = [case_number["Date"] for case_number in cases_response]

# Create dataframe and include month column and set to the month specified in url 
march_covid_df = pd.DataFrame({
         "Confirmed (cumulative)": confirmed,
         "Active": active,
         "Deaths": deaths,
         "State": state,
         "Date": date,
         "Month": "March"
    
 }) 

In [108]:
# Covid-19 API request for Australia, case status includes active, confirmed, deaths, recovered
# Month: April

# Build URL using the Covid-19 api
url = "https://api.covid19api.com/country/australia?from=2020-04-01T00:00:00Z&to=2020-04-30T00:00:00Z"

# Make request
response = requests.get(url)

# Convert to json
cases_response = response.json()

# Use list comprehension to get case numbers
confirmed = [case_number["Confirmed"] for case_number in cases_response]
active = [case_number["Active"] for case_number in cases_response]
deaths = [case_number["Deaths"] for case_number in cases_response]
state = [case_number["Province"] for case_number in cases_response]
date = [case_number["Date"] for case_number in cases_response]

# Create dataframe and include month column and set to the month specified in url 
april_covid_df = pd.DataFrame({
         "Confirmed (cumulative)": confirmed,
         "Active": active,
         "Deaths": deaths,
         "State": state,
         "Date": date,
         "Month": "April"
    
 }) 


In [109]:
# Covid-19 API request for Australia, case status includes active, confirmed, deaths, recovered
# Month: May

# Build URL using the Covid-19 api
url = "https://api.covid19api.com/country/australia?from=2020-05-01T00:00:00Z&to=2020-05-31T00:00:00Z"

# Make request
response = requests.get(url)

# Convert to json
cases_response = response.json()

# Use list comprehension to get case numbers
confirmed = [case_number["Confirmed"] for case_number in cases_response]
active = [case_number["Active"] for case_number in cases_response]
deaths = [case_number["Deaths"] for case_number in cases_response]
state = [case_number["Province"] for case_number in cases_response]
date = [case_number["Date"] for case_number in cases_response]

# Create dataframe and include month column and set to the month specified in url 
may_covid_df = pd.DataFrame({
         "Confirmed (cumulative)": confirmed,
         "Active": active,
         "Deaths": deaths,
         "State": state,
         "Date": date,
         "Month": "May"
    
 }) 

In [110]:
# Covid-19 API request for Australia, case status includes active, confirmed, deaths, recovered
# Month: June

# Build URL using the Covid-19 api
url = "https://api.covid19api.com/country/australia?from=2020-06-01T00:00:00Z&to=2020-06-30T00:00:00Z"

# Make request
response = requests.get(url)

# Convert to json
cases_response = response.json()

# Use list comprehension to get case numbers
confirmed = [case_number["Confirmed"] for case_number in cases_response]
active = [case_number["Active"] for case_number in cases_response]
deaths = [case_number["Deaths"] for case_number in cases_response]
state = [case_number["Province"] for case_number in cases_response]
date = [case_number["Date"] for case_number in cases_response]

# Create dataframe and include month column and set to the month specified in url 
june_covid_df = pd.DataFrame({
         "Confirmed (cumulative)": confirmed,
         "Active": active,
         "Deaths": deaths,
         "State": state,
         "Date": date,
         "Month": "June"
    
 }) 

In [113]:
# Covid-19 API request for Australia, case status includes active, confirmed, deaths, recovered
# Month: July

# Build URL using the Covid-19 api
url = "https://api.covid19api.com/country/australia?from=2020-07-01T00:00:00Z&to=2020-07-31T00:00:00Z"

# Make request
response = requests.get(url)

# Convert to json
cases_response = response.json()

# Use list comprehension to get case numbers
confirmed = [case_number["Confirmed"] for case_number in cases_response]
active = [case_number["Active"] for case_number in cases_response]
deaths = [case_number["Deaths"] for case_number in cases_response]
state = [case_number["Province"] for case_number in cases_response]
date = [case_number["Date"] for case_number in cases_response]

# Create dataframe and include month column and set to the month specified in url 
july_covid_df = pd.DataFrame({
         "Confirmed (cumulative)": confirmed,
         "Active": active,
         "Deaths": deaths,
         "State": state,
         "Date": date,
         "Month": "July"
    
 }) 


In [118]:
# Covid-19 API request for Australia, case status includes active, confirmed, deaths, recovered
# Month: August

# Build URL using the Covid-19 api
url = "https://api.covid19api.com/country/australia?from=2020-06-01T00:00:00Z&to=2020-06-30T00:00:00Z"

# Make request
response = requests.get(url)

# Convert to json
cases_response = response.json()

# Use list comprehension to get case numbers
confirmed = [case_number["Confirmed"] for case_number in cases_response]
active = [case_number["Active"] for case_number in cases_response]
deaths = [case_number["Deaths"] for case_number in cases_response]
state = [case_number["Province"] for case_number in cases_response]
date = [case_number["Date"] for case_number in cases_response]

# Create dataframe and include month column and set to the month specified in url 
august_covid_df = pd.DataFrame({
         "Confirmed (cumulative)": confirmed,
         "Active": active,
         "Deaths": deaths,
         "State": state,
         "Date": date,
         "Month": "August"
    
 }) 

In [133]:
# Concat dataframes
mar_to_aug_covid_df = pd.concat([march_covid_df, april_covid_df, may_covid_df, june_covid_df, july_covid_df, august_covid_df])
mar_to_aug_covid_df.to_csv("census_data_states.csv", encoding="utf-8")

In [130]:
# cumulative cases by (end of) month and state
# mar_cumulative = mar_to_aug_covid_df.loc[mar_to_aug_covid_df["Date"] == "2020-03-31T00:00:00Z"]
# apr_cumulative = mar_to_aug_covid_df.loc[mar_to_aug_covid_df["Date"] == "2020-04-30T00:00:00Z"]
# may_cumulative = mar_to_aug_covid_df.loc[mar_to_aug_covid_df["Date"] == "2020-05-31T00:00:00Z"]
# jun_cumulative = mar_to_aug_covid_df.loc[mar_to_aug_covid_df["Date"] == "2020-06-30T00:00:00Z"]
# jul_cumulative = mar_to_aug_covid_df.loc[mar_to_aug_covid_df["Date"] == "2020-07-31T00:00:00Z"]
# aug_cumulative = mar_to_aug_covid_df.loc[mar_to_aug_covid_df["Date"] == "2020-08-31T00:00:00Z"]

,Confirmed (cumulative),Active,Deaths,State,Date,Month
240,80,76,1,Australian Capital Territory,2020-03-31T00:00:00Z,March
241,2032,2020,8,New South Wales,2020-03-31T00:00:00Z,March
242,17,17,0,Northern Territory,2020-03-31T00:00:00Z,March
243,743,733,2,Queensland,2020-03-31T00:00:00Z,March
244,337,331,0,South Australia,2020-03-31T00:00:00Z,March
245,69,63,1,Tasmania,2020-03-31T00:00:00Z,March
246,917,622,4,Victoria,2020-03-31T00:00:00Z,March
247,364,321,2,Western Australia,2020-03-31T00:00:00Z,March


In [137]:
inputFormatter = DateTimeFormatter.ofPattern("yyyy-MM-dd'T'HH:mm:ss.SSS'Z'", Locale.ENGLISH);
outputFormatter = DateTimeFormatter.ofPattern("dd-MM-yyy", Locale.ENGLISH);
date = LocalDate.parse("2018-04-10T04:00:00.000Z", inputFormatter);
formattedDate = outputFormatter.format(date);
System.out.println(formattedDate);

NameError: name 'DateTimeFormatter' is not defined